In [1]:
import boto3
import os

# تنظیمات اتصال به Arvan Object Storage
access_key = "a8761df0-960e-4dd1-b5f2-ef8ef60823a9"
secret_key = "f502aad1cec94636d4381cadf302a6114df05bf825864e554b82010d6d2441ab"
region_name = "s3.ir-thr-at1.arvanstorage.ir"
bucket_name = "ehsannima"
folder_name = "xray/"  # پوشه تصاویر داخل باکت

# اتصال
s3 = boto3.client(
    "s3",
    endpoint_url=f"https://{region_name}",
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)

# مسیر ذخیره تصاویر به صورت محلی
save_path = "./xray_images"
os.makedirs(save_path, exist_ok=True)

# لیست و دانلود تصاویر
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_name)

for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.endswith((".jpg", ".png", ".jpeg")):
        filename = os.path.basename(key)
        file_path = os.path.join(save_path, filename)
        s3.download_file(bucket_name, key, file_path)
        print(f"✅ Downloaded: {filename}")


✅ Downloaded: 100(1).jpg
✅ Downloaded: 100.jpg
✅ Downloaded: 1000.jpg
✅ Downloaded: 1001.jpg
✅ Downloaded: 1002(1).jpg
✅ Downloaded: 1002.jpg
✅ Downloaded: 1003(1).jpg
✅ Downloaded: 1003.jpg
✅ Downloaded: 1004(1).jpg
✅ Downloaded: 1004.jpg
✅ Downloaded: 1005.jpg
✅ Downloaded: 1006.jpg
✅ Downloaded: 1007.jpg
✅ Downloaded: 1008.jpg
✅ Downloaded: 1009.jpg
✅ Downloaded: 101.jpg
✅ Downloaded: 1010.jpg
✅ Downloaded: 1011.jpg
✅ Downloaded: 1012.jpg
✅ Downloaded: 1013.jpg
✅ Downloaded: 1014.jpg
✅ Downloaded: 1015.jpg
✅ Downloaded: 1016.jpg
✅ Downloaded: 1017.jpg
✅ Downloaded: 1018.jpg
✅ Downloaded: 1019.jpg
✅ Downloaded: 102.jpg
✅ Downloaded: 1020.jpg
✅ Downloaded: 1021.jpg
✅ Downloaded: 1022.jpg
✅ Downloaded: 1023.jpg
✅ Downloaded: 1024.jpg
✅ Downloaded: 1025.jpg
✅ Downloaded: 1026.jpg
✅ Downloaded: 103.jpg
✅ Downloaded: 104.jpg
✅ Downloaded: 105.jpg
✅ Downloaded: 106.jpg
✅ Downloaded: 107.jpg
✅ Downloaded: 108.jpg
✅ Downloaded: 109.jpg
✅ Downloaded: 110.jpg
✅ Downloaded: 111.jpg
✅ Downloaded

In [ ]:
!pip install 

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.0 MB ? eta -:--:--
   - -------------------------------------- 1.3/39.0 MB 4.8 MB/s eta 0:00:08
   - -------------------------------------- 1.6/39.0 MB 5.2 MB/s eta 0:00:08
   -- ------------------------------------- 2.1/39.0 MB 2.9 MB/s eta 0:00:13
   --- ------------------------------------ 3.7/39.0 MB 4.0 MB/s eta 0:00:09
   ---- ----------------------------------- 4.7/39.0 MB 4.3 MB/s eta 0:00:09
   ----- ---------------------------------- 5.8/39.0 MB 4.3 MB/s eta 0:00:08
   ------ --------------------------------- 6.8/39.0 MB 4.5 MB/s eta 0:00:08
   -------- ------------------------------- 7.9/39.0 MB 4.5 MB/s eta 0:00:07
   -------- ------------------------------- 8.7/39.0 MB 4.5 MB/s eta 0:00:07
   --------- ------------------------------ 9.4/39.0 MB 4.4 MB/s eta 0:00:07
   ----------- -----


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
